se realiza el analsis a partir de la textos tokenizados

In [ ]:
import pandas as pd


df = pd.read_excel('/content/DATA_ejercicio.xlsx')
#se tomo la data pero se toquenizo para realizar el ejercicio

In [ ]:
# Renombramos las columnas mediante .columns de pandas
df.columns = ['codigo','texto']

# visualizar últimos registros del dataset mediante .tail de pandas
print(df.head())

                codigo                                              texto
0  1542289655323835904  ['qué', 'horror', 'hallan', 'enterrada', 'en',...
1  1542261143233860096  ['‘putin', 'no', 'habría', 'invadido', 'ucrani...
2  1542113466261164032  ['una', 'colombiana', 'entre', 'las', 'mujeres...
3  1541648018516200960  ['la', 'mujer', 'que', 'tuvo', 'una', 'violent...
4  1541117721168994048  ['la', 'mujer', 'saltó', 'la', 'piscina', 'al'...


In [ ]:
!pip install transformers pandas torch

In [ ]:

import nltk
from nltk.corpus import stopwords
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Descargar las stopwords en español
nltk.download('stopwords')
spanish_stopwords = set(stopwords.words('spanish'))

# Asegurarse de que los tokens estén en formato de listas al cargar los datos
df['texto'] = df['texto'].apply(eval)

# Función para limpiar los tokens eliminando stopwords en caso de que no se realizo
def clean_tokens(tokens):
    # Filtrar los tokens que no son stopwords
    cleaned_tokens = [token for token in tokens if token.lower() not in spanish_stopwords]
    return cleaned_tokens

# Aplicar la limpieza a la columna de tokenización
df['cleaned_tokens'] = df['texto'].apply(clean_tokens)
#_______________________________________________________________________________
# Cargar el modelo y el tokenizador preentrenado para análisis de sentimientos puede usar varios idiomas, clasifica en 5 categorias
# este modelo es mas pesado
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Función para predecir el sentimiento
def predict_sentiment(tokens):
    # Unir los tokens limpios en una sola cadena de texto
    text = " ".join(tokens)
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    logits = outputs.logits
    sentiment = torch.argmax(logits, dim=1).item()
    # El modelo tiene 5 clases: 0 - Muy negativo, 1 - Negativo, 2 - Neutro, 3 - Positivo, 4 - Muy positivo
    if sentiment == 0:
        return "Muy Negativo"
    elif sentiment == 1:
        return "Negativo"
    elif sentiment == 2:
        return "Neutro"
    elif sentiment == 3:
        return "Positivo"
    else:
        return "Muy Positivo"
#____________________________________________________________________
# Aplicar la función de predicción a la columna de tokens limpios
df['Sentiment'] = df['cleaned_tokens'].apply(predict_sentiment)

# Mostrar las primeras filas del dataset con la nueva columna de sentimientos
df.head()
# es

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

,codigo,texto,cleaned_tokens,Sentiment
0,1542289655323835904,"[qué, horror, hallan, enterrada, en, su, patio...","[horror, hallan, enterrada, patio, joven, llev...",Muy Negativo
1,1542261143233860096,"[‘putin, no, habría, invadido, ucrania, si, fu...","[‘putin, invadido, ucrania, si, mujer’, boris,...",Muy Negativo
2,1542113466261164032,"[una, colombiana, entre, las, mujeres, más, po...","[colombiana, mujeres, poderosas, negocios, méx...",Muy Positivo
3,1541648018516200960,"[la, mujer, que, tuvo, una, violenta, reacción...","[mujer, violenta, reacción, oficina, claro, pa...",Muy Negativo
4,1541117721168994048,"[la, mujer, saltó, la, piscina, al, rescate, d...","[mujer, saltó, piscina, rescate, discípula, hu...",Muy Negativo


In [ ]:
df.to_excel('/content/df_exportado.xlsx', index=False)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

#___________________________________________________
# tambien se podria usar este modelo diseñado para solo español, es menos pesado
# este modelo solo tiene 3 polaridades
model_name = "pysentimiento/robertuito-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Función para predecir sentimiento
def predict_sentiment_robertuito(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    labels = ["NEG", "NEU", "POS"]
    return labels[prediction]
#_______________________________________________________
